In [ ]:
# ?꾩쿂由?諛??꾩쿂由?異붽?
import cv2
import torch
import pandas
import numpy as np

import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2

import ncnn
import RPi.GPIO as GPIO

IMG_SIZE = 320 # resize size
CONF_TH = 0.6
FRAME_SKIP = 2
IOU_TH = 0.25
DETECT_PADDING = 30

COLOR_RED = (0,0,255)
COLOR_GREEN = (0,255,0)
COLOR_BLUE = (255,0,0)

handle_pixel = []
blade_pixel = []

# 洹몃━湲?def draw_landmarks_on_image(tools_dect, hand_dect, frame):
    landmarks_list = hand_dect.hand_landmarks
    annotated_image = np.copy(frame)

    # ?꾧뎄 諛뺤뒪 洹몃━湲?    for (x1,y1,x2,y2), s, _ in tools_dect:
        cv2.rectangle(annotated_image, (x1,y1), (x2,y2), (0,0,255), 2)

    # ???쒕뱶留덊겕 洹몃━湲?    for landmark in landmarks_list:
        landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        for lm in landmark: 
            landmarks_proto.landmark.extend([landmark_pb2.NormalizedLandmark(x=lm.x, y=lm.y, z=lm.z)])
            
        solutions.drawing_utils.draw_landmarks(
            annotated_image,
            landmarks_proto,
            solutions.hands.HAND_CONNECTIONS,
            solutions.drawing_styles.get_default_hand_landmarks_style()
        )

    return annotated_image

# 寃뱀묩 ?뺤씤
# xyxy: x, y, ?덈퉬, ?믪씠
# aknife : 0; blade: 1; handle: 2
def detection_box(tools_dect, hand_dect, frame):
    annotated_image = np.copy(frame)
        
    # boolean 留덉뒪??    # ?좉낵 ?먯옟?댁씤 寃쎌슦留?媛?몄샂
    blade_xy = [ tools[0] for tools in tools_dect if tools[2] == 0 ]
    handle_xy = [ tools[0] for tools in tools_dect if tools[2] == 1 ]
    
    # ?댁쟾 ?꾨젅??醫뚰몴 ???    global handle_pixel, blade_pixel
    if len(handle_xy) > 0:
        handle_pixel = save_pixel(handle_xy)
        
    if len(blade_xy) > 0:
        blade_pixel = save_pixel(blade_xy)

    # ?먯씠 寃뱀낀?붿? ?뺤씤
    hand_list = hand_dect.hand_landmarks
    H, W, _ = annotated_image.shape
    
    flag_blade = False; flag_handle = False
    flag_Up = False; flag_Down = False; flag_Left = False; flag_Right = False
    for hand in hand_list:
        middle_x = int(hand[9].x * W)
        middle_y = int(hand[9].y * H)
        cv2.circle(annotated_image, (middle_x, middle_y), 5, (255,255,0), -1, cv2.LINE_AA)
        
        # ?먯옟???뺤씤
        for handle in handle_pixel:
            handle_middle_x = (handle[0] + handle[2]) // 2
            handle_middle_y = (handle[1] + handle[3]) // 2
            cv2.circle(annotated_image, (handle_middle_x, handle_middle_y), 3, (255,255,0), -1, cv2.LINE_AA)

            if check_inside(handle, middle_x, middle_y):
                flag_handle = True
                break

            # show_derection
            if middle_x < handle_middle_x-DETECT_PADDING: flag_Right = True
            elif middle_x > handle_middle_x+DETECT_PADDING: flag_Left = True
                    
            if middle_y < handle_middle_y-DETECT_PADDING: flag_Down = True
            elif middle_y > handle_middle_y+DETECT_PADDING: flag_Up = True
            
        
        for lm in hand:
            lm_x = int(lm.x * W)
            lm_y = int(lm.y * H)
            
            # ???뺤씤
            for blade in blade_pixel:
                if check_inside(blade, lm_x, lm_y):
                    flag_blade = True
                    break

    
                    
    # 10: Right; 22: Left; 17: Up; 27: Down
    # 16: Left-Up; 21: Right-Up; 20: Right-Down; 12:Left-Down
    # 5: center
    false_LED()
    if flag_Left:
        if flag_Up: GPIO.output(16, True)
        elif flag_Down: GPIO.output(21, True)
        else: GPIO.output(10, True)
    elif flag_Right:
        if flag_Up: GPIO.output(12, True)
        elif flag_Down: GPIO.output(16, True)
        else: GPIO.output(22, True)
    else:
        if flag_Up: GPIO.output(27, True)
        elif flag_Down: GPIO.output(17, True)

    if flag_blade:
        if flag_handle:
            detected_LED()
        else:
            GPIO.output(9, True)
    elif flag_handle:
        detected_LED()
                                
    return annotated_image

def save_pixel(boxes):
    lr = []
    for x1,y1,x2,y2 in boxes:
        lr.append([x1,y1,x2,y2])
    return lr

def check_inside(box, x, y):
    if box[0] <= x <= box[2] and box[1] <= y <= box[3]: 
        return True
    else: return False

def false_LED():
    GPIO.output(10,False); GPIO.output(22,False)
    GPIO.output(17,False); GPIO.output(27,False)

    GPIO.output(21,False); GPIO.output(20,False)
    GPIO.output(16,False); GPIO.output(12,False)
    GPIO.output(9,False)

def detected_LED():
    GPIO.output(10,True); GPIO.output(22,True)
    GPIO.output(17,True); GPIO.output(27,True)

    GPIO.output(21,True); GPIO.output(20,True)
    GPIO.output(16,True); GPIO.output(12,True)

# ?꾩쿂由?def letterbox(img, new=IMG_SIZE, color=(114,114,114)):
    h, w = img.shape[:2]
    r = min(new / h, new / w)
    nh, nw = int(round(h * r)), int(round(w * r))
    resized = cv2.resize(img, (nw, nh), interpolation=cv2.INTER_AREA)
    canvas = np.full((new, new, 3), color, dtype=np.uint8)
    top = (new - nh) // 2
    left = (new - nw) // 2
    canvas[top:top+nh, left:left+nw] = resized
    return canvas, r, left, top

def nms(dets, iou_th=IOU_TH):
    # dets: [([x1,y1,x2,y2], score, cls_id), ...] in letterbox 醫뚰몴
    if not dets: return []
    boxes = np.array([d[0] for d in dets], dtype=np.float32)
    scores = np.array([d[1] for d in dets], dtype=np.float32)
    order = scores.argsort()[::-1]
    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        if order.size == 1: break
        rest = order[1:]
        xx1 = np.maximum(boxes[i,0], boxes[rest,0])
        yy1 = np.maximum(boxes[i,1], boxes[rest,1])
        xx2 = np.minimum(boxes[i,2], boxes[rest,2])
        yy2 = np.minimum(boxes[i,3], boxes[rest,3])
        inter = np.maximum(0.0, xx2 - xx1) * np.maximum(0.0, yy2 - yy1)
        area_i = (boxes[i,2] - boxes[i,0]) * (boxes[i,3] - boxes[i,1])
        area_r = (boxes[rest,2] - boxes[rest,0]) * (boxes[rest,3] - boxes[rest,1])
        iou = inter / (area_i + area_r - inter + 1e-6)
        order = rest[iou <= iou_th]
    return [dets[k] for k in keep]

def tools_inference(frame):
    H, W, _ = frame.shape
    img_lbx, r, lpad, tpad = letterbox(frame, IMG_SIZE)
    img_rgb = cv2.cvtColor(img_lbx, cv2.COLOR_BGR2RGB)
    
    input_mat = ncnn.Mat.from_pixels(img_rgb, ncnn.Mat.PixelType.PIXEL_RGB, IMG_SIZE, IMG_SIZE)
    input_mat.substract_mean_normalize([0,0,0], [1/255.0, 1/255.0, 1/255.0])

    # 異붾줎 ?쒖옉
    ex = net.create_extractor()
    ex.input("in0", input_mat)
    _, result = ex.extract("out0")

    arr = result.numpy()
    D, N = arr.shape
    A = arr.T  # (N, D) : ???됱씠 ???꾨낫
    cx, cy, w, h = A[:, 0], A[:, 1], A[:, 2], A[:, 3]

    cls_scores = A[:, 4:7]              # (N,3)
    cls_ids = np.argmax(cls_scores, axis=1)
    scores  = cls_scores[np.arange(N), cls_ids]
    
    # ?몄떇瑜좊줈 媛?몄삤湲?    keep = scores >= CONF_TH
    if not np.any(keep): return []
    cx = cx[keep]; cy = cy[keep]; w = w[keep]; h = h[keep]
    scores = scores[keep]; cls_ids = cls_ids[keep]

    # cxcywh -> xyxy (?덊꽣諛뺤뒪??IMG_SIZE 湲곗?)
    x1 = cx - w/2; y1 = cy - h/2
    x2 = cx + w/2; y2 = cy + h/2

    # ?먮낯 醫뚰몴濡??????    x1 = (x1 - lpad) / r; y1 = (y1 - tpad) / r
    x2 = (x2 - lpad) / r; y2 = (y2 - tpad) / r

    # ?대━??    x1 = np.clip(x1, 0, W); y1 = np.clip(y1, 0, H)
    x2 = np.clip(x2, 0, W); y2 = np.clip(y2, 0, H)

    dets = [(
        [int(x1[i]), int(y1[i]), int(x2[i]), int(y2[i])], 
        float(scores[i]), int(cls_ids[i])
        ) 
        for i in range(len(scores))
    ]

    dets = nms(dets, IOU_TH)

    return dets
# ===================================================
# ???쒕뱶留덊겕 紐⑤뜽 媛?몄삤湲?base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options, num_hands=1)
hand_detector = vision.HandLandmarker.create_from_options(options)
# ===================================================
# 移?援щ텇 紐⑤뜽 媛?몄삤湲?(ncnn ?ㅽ듃)
net = ncnn.Net()
net.opt.num_threads = 4
net.opt.use_fp16_storage = True
net.opt.use_fp16_arithmetic = True

net.load_param("knife_total_320.ncnn.param")
net.load_model("knife_total_320.ncnn.bin")
# ===================================================

# 罹??닿린
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("??罹좎쓣 ?????놁뒿?덈떎")
    exit()

fid = 0
GPIO.setmode(GPIO.BCM)
GPIO.setup(10,GPIO.OUT); GPIO.setup(22,GPIO.OUT); GPIO.setup(17,GPIO.OUT); GPIO.setup(27,GPIO.OUT)
GPIO.setup(21,GPIO.OUT); GPIO.setup(20,GPIO.OUT); GPIO.setup(16,GPIO.OUT); GPIO.setup(12,GPIO.OUT)
GPIO.setup(9,GPIO.OUT)

# ?ㅽ뻾遺
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_bgr = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_bgr)

    # ?꾨젅???ㅽ궢
    fid += 1
    if fid % (FRAME_SKIP + 1) == 1:
        tools_result = tools_inference(frame)
        hand_result = hand_detector.detect(mp_image)

    # 寃곌낵 洹몃━湲?    drawing_image = draw_landmarks_on_image(tools_result, hand_result, frame)
    annotated_image = detection_box(tools_result, hand_result, drawing_image)

    cv2.imshow("hand_land", annotated_image)

    # q?ㅻ? ?꾨Ⅴ硫?醫낅즺
    if cv2.waitKey(1) & 0xFF == ord("q"):
        false_LED()
        break
    
# 醫낅즺 ???먯썝 ?댁젣
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)